In [2]:
import glob
import os
import json
import pandas as pd
from tqdm import tqdm
import time
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from scripts.report_agent import ReportAgent
from typing import List

load_dotenv()

d:\ceo-extraction-multi-turn\ceo-extraction-multi-turn\.venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


True

In [4]:
# Fungsi untuk membuat direktori jika belum ada
def create_output_directory(output_path: str):
    """
    Ensure that the output directory exists.
    """
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Fungsi untuk memproses satu putaran data (first turn atau second turn)
def process_turn(query_prompt: str, schema_class: BaseModel, input_dir: str, output_dir: str, input_file: str = None, glob_path: str = "*/*.json"):
    """
    Process a single round of data extraction (first or second turn) and save the results to the specified output directory.
    
    Args:
    - query_prompt (str): The query prompt for the report extraction.
    - schema_class (BaseModel): The schema class for validating the extracted data.
    - input_dir (str): The directory containing input JSON files.
    - output_dir (str): The directory where the results will be saved.
    - input_file (str, optional): If provided, the function will filter data based on this file (e.g., the first turn results).
    - glob_path (str, optional): The glob pattern to match input files. Default is "*/*.json".
    """
    # Create the output directory if it does not exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Define the list of files to process based on the glob pattern
    files = glob.glob(os.path.join(input_dir, glob_path))
    
    # Check if files exist, and raise an error if not
    if not files:
        raise FileNotFoundError(f"No files matched the glob pattern: '{glob_path}' in directory '{input_dir}'")
    
    # Process each file
    for file in tqdm(files, desc="Processing Companies"):
        try:
            company = os.path.splitext(os.path.basename(file))[0]
            agent = ReportAgent(os.environ["LLM"], schema_class, query_prompt)
            
            # Handle data processing for first or second turn
            if input_file:
                # For the second turn, filter data based on the input Excel file
                fdata = pd.read_excel(input_file)
                fdata = fdata[fdata['company'] == company]
                fdata = fdata.drop("sources", axis=1)
                
                # Loop through each row of filtered data
                for idx, row in fdata.iterrows():
                    prompt_data = row.to_dict()
                    result = agent.get_data(prompt_data)
                    result['data'].update(prompt_data)
                    
                    # Generate output path and save result
                    relative_path = os.path.relpath(file, input_dir)
                    output_path = os.path.join(output_dir, relative_path).replace(".json", f"_{idx+1}.json")
                    create_output_directory(output_path)
                    with open(output_path, 'w') as f:
                        json.dump(result, f, indent=4)
            else:
                # For the first turn, directly use the company data
                result = agent.get_data({'company': company})
                result['data']['company'] = company
                
                # Generate output path and save result
                relative_path = os.path.relpath(file, input_dir)
                output_path = os.path.join(output_dir, relative_path)
                create_output_directory(output_path)
                with open(output_path, 'w') as f:
                    json.dump(result, f, indent=4)
                    
        except Exception as e:
            print(f"\nError processing {company}: {str(e)}")
        
        # Pause to avoid overloading the API
        time.sleep(30)
    
    # After processing all files, save results to an Excel file
    save_results_to_excel(output_dir)

# Fungsi untuk menyimpan hasil pemrosesan ke dalam file Excel
def save_results_to_excel(output_dir: str):
    """
    Save the processed results into an Excel file after extraction.
    """
    output_files = glob.glob(os.path.join(output_dir, "*", "*.json"))
    output_list = []
    
    for file in output_files:
        with open(file, 'r') as f:
            result = json.load(f)
        result['data'].update({'sources': result['sources']})
        output_list.append(result['data'])
    
    output_df = pd.DataFrame(output_list)
    output_df.to_excel(f"{output_dir}/result.xlsx", index=False)

### Start here

First Turn
- Siapkan query prompt dengan placeholder {company}
- Siapkan schema / variabel yang ingin di-crawl
- Siapkan `input_dir`, `output_dir`, dan `glob_path`

    `input_dir` dan `glob_path` saling berhubungan. Input direktori adalah tempat file json disimpan.

    Misalnya, file json terletak di path `data/JSON/AR/ID_ADRO_AR_2022.json`. 

    Maka `input_dir` bisa berupa `data`, lalu `glob_path`-nya adalah `*/*/*.json`
    ATAU jika `input_dir` berupa `data/JSON`, lalu `glob_path`-nya adalah `*/*.json`

## Phase 1

In [ ]:
# # FIRST TURN RAW

# # QUERY PROMPT
# query_prompt_1 = """
# Retrieve the name of the BOD (board of director) and BOC (Board of Company) for {company} from the company's report.
# You may use detailed queries for the retriever tool multiple times to ensure the information is accurate.
# """

# # SCHEMA
# class GovernanceBody(BaseModel):
#     ceo_name: str = Field(description="Nama CEO atau presiden direktur dari perusahaan", example="Maula Irfani")

# # Proses first turn
# process_turn(
#     query_prompt=query_prompt_1,
#     schema_class=GovernanceBody,
#     input_dir="data/JSON",
#     output_dir="results_1st_turn/CG REPORT",
#     glob_path="*/*.json"
# )

In [ ]:
# FIRST TURN

# QUERY PROMPT
query_prompt_1 = """
Retrieve the name of the BOD (board of director) and BOC (Board of Company) for {company} from the company's report.
You may use detailed queries for the retriever tool multiple times to ensure the information is accurate.
"""

# SCHEMA
class BoardEntry(BaseModel):
    name: str = Field(description="Nama dari BOD atau BOC dari perusahaan", example="Giovanny Alberta")
    position: str = Field(description="Posisi atau jabatan dari orang tersebut pada perusahaan", example="Direktur Marketing")

class GovernanceBody(BaseModel):
    bod: List[BoardEntry] = Field(description="Seluruh orang yang menjadi BOD dari perusahaan lengkap dengan jabatannya")
    boc: List[BoardEntry] = Field(description="Seluruh orang yang menjadi BOC dari perusahaan lengkap dengan jabatannya")

# Proses first turn
process_turn(
    query_prompt=query_prompt_1,
    schema_class=GovernanceBody,
    input_dir="data/JSON",
    output_dir="results_1st_turn/CG REPORT",
    glob_path="*/*.json"
)

Processing Companies:   0%|          | 0/11 [00:00<?, ?it/s]



> Entering new Invoke Agent chain...

Invoking: `retriever_tool` with `{'query': 'Please provide the names of the members of the Board of Directors (BOD) for ID_ADRO_AR_2022.'}`


[]
Invoking: `retriever_tool` with `{'query': 'Please provide the names of the members of the Board of Commissioners (BOC) for ID_ADRO_AR_2022.'}`


[]
Invoking: `retriever_tool` with `{'query': 'List the Board of Directors (BOD) from the 2022 Annual Report of ADRO.'}`


[]
Invoking: `retriever_tool` with `{'query': 'List the Board of Commissioners (BOC) from the 2022 Annual Report of ADRO.'}`


[]
Invoking: `retriever_tool` with `{'query': 'Who are the members of the Board of Directors (BOD) in the Annual Report of ADRO for 2022?'}`


[]
Invoking: `retriever_tool` with `{'query': 'Who are the members of the Board of Commissioners (BOC) in the Annual Report of ADRO for 2022?'}`


[]It appears that I am unable to retrieve the names of the Board of Directors (BOD) and Board of Commissioners (BOC) for ID_ADRO_

Processing Companies:   9%|▉         | 1/11 [01:03<10:30, 63.10s/it]



> Entering new Invoke Agent chain...

Invoking: `retriever_tool` with `{'query': 'Name of Board of Directors (BOD) for ID_ABBA_IR_2023'}`


[Document(id='cf676a46-4c7e-44dc-9c32-812ed5ac92b6', metadata={'company': 'ID_ABBA_IR_2023', 'page': 136.0}, page_content='bagi Direksi. Hasil evaluasi kinerja Direktur secara  individual  merupakan  salah  satu  dasar pertimbangan untuk pemberhentian dan/atau menunjuk kembali Direktur yang bersangkutan.\n\n## Susunan Direksi\n\nPada  tahun  2023,  susunan  Direksi  Perseroan tidak  mengalami  perubahan,  sehingga  susunan Direksi Perseroan adalah sebagai berikut:\n\nthe results of individual performance are one of the  basic  considerations  for  dismissal  and/or reappointment of the concerned Director.\n\n## Board Of Directors\n\nIn  2023,  there  was  no  change  in  the  Board  of Directors composition. Thus, the composition is as follows:\n\nTugas  dan  Tanggung    Jawab  Masing-Masing Anggota Direksi\n\n| Nama  Name                        

Processing Companies:  18%|█▊        | 2/11 [01:49<07:57, 53.07s/it]

...

Processing Companies:  82%|████████▏ | 9/11 [07:33<01:39, 49.85s/it]



> Entering new Invoke Agent chain...

Invoking: `retriever_tool` with `{'query': 'BOD members for ID_AGRS_IR_2023'}`


[Document(id='38c296bb-b663-467b-9134-771c6433f267', metadata={'company': 'ID_AGRS_IR_2023', 'page': 179.0}, page_content="pelatihan anggota Komite Remunerasi dan Nominasi lainnya sepanjang tahun 2023 diuraikan sebagai berikut:\n\n<!-- image -->\n\ntraining programs for other Remuneration and Nomination Committee members throughout 2023 were described as follows:\n\n| Nama Name       | Topik Pelatihan Training Topic                                                                                                                                                                                         | Tanggal Date                              | Penyelenggara Organizer                                                                           |\n|-----------------|-------------------------------------------------------------------------------------------------------------

Processing Companies:  91%|█████████ | 10/11 [08:26<00:50, 50.87s/it]



> Entering new Invoke Agent chain...

Invoking: `retriever_tool` with `{'query': 'name of the Board of Directors (BOD) for ID_AIMS_IR_2023'}`


[Document(id='701469ee-c7c7-4714-ad72-42c7beda7486', metadata={'company': 'ID_AIMS_IR_2023', 'page': 51.0}, page_content="<!-- image -->\n\n## PERUBAHAN SUSUNAN DIREKSI TAHUN 2023 CHANGES TO THE BOARD OF DIRECTORS' COMPOSITION IN 2023\n\n<!-- image -->\n\nBerdasarkan keputusan RUPS Tahunan yang berlangsung pada 28 Juni 2023 dan RUPS Luar Biasa pada 19 Oktober 2023, pemegang saham menyetujui perubahan susunan Direksi Perseroan. Berikut disampaikan  kronologi  perubahan  susunan  Direksi Perseroan  di  sepanjang  tahun  2023  dan  susunan Direksi Perseroan  per 31 Desember 2023:\n\nBased on the decisions of the Annual GMS held on June 28, 2023, and the Extraordinary GMS on October 19, 2023, the shareholders approved changes to the composition  of  the  Company's  Board  of  Directors. Here is the chronology of changes to the composition of  the

Processing Companies: 100%|██████████| 11/11 [09:13<00:00, 50.31s/it]


## Phase 2

- Siapkan query prompt yang memiliki placeholder sesuai variabel yang ingin dicari, dimana variabel tersebut seharusnya sudah didapatkan dari first turn.
- Masukkan parameter tambahan yaitu `input_file` yang merupakan hasil dari path excel hasil dari first turn

In [ ]:
# # SECOND TURN RAW

# # QUERY PROMPT
# query_prompt_2 = """
# Retrieve the bachelor education degree of {ceo_name} which is ceo of {company} from the company's report. 
# """

# # SCHEMA
# class EducationEntry(BaseModel):
#     major: str = Field(description="The academic program or field of study pursued by the ceo.", example="Business Administration")
#     university: str = Field(description="The name of the university where the ceo pursued their education.", example="Harvard University")
#     country: str = Field(description="The country where the ceo completed their education.", example="United States")

# # Proses second turn
# process_turn(
#     query_prompt=query_prompt_2,
#     schema_class=EducationEntry,
#     input_dir="data/JSON",
#     output_dir="results_2nd_turn",
#     input_file="results_1st_turn/result.xlsx",
#     glob_path="*/*.json"
# )

Processing Companies:   0%|          | 0/11 [00:00<?, ?it/s]


Error processing ID_ADRO_AR_2022: string indices must be integers, not 'str'


Processing Companies:   9%|▉         | 1/11 [00:30<05:09, 30.91s/it]


Error processing ID_ABBA_IR_2023: string indices must be integers, not 'str'


Processing Companies:  18%|█▊        | 2/11 [01:01<04:35, 30.66s/it]


Error processing ID_ABDA_IR_2023: string indices must be integers, not 'str'


Processing Companies:  27%|██▋       | 3/11 [01:32<04:05, 30.65s/it]


Error processing ID_ACRO_IR_2023: string indices must be integers, not 'str'


In [5]:
import json
import os
import pandas as pd
from tqdm import tqdm

# Load hasil dari phase 1 (Excel yang berisi BOD dan BOC)
df_phase_1 = pd.read_excel("results_1st_turn/CG REPORT/result.xlsx")

# SCHEMA
class EducationEntry(BaseModel):
    name: str = Field(description="Nama individu yang memiliki latar belakang pendidikan", example="Giovanny Alberta")
    position: str = Field(description="Posisi individu di perusahaan", example="Director")
    major: str = Field(description="Program akademik atau jurusan yang diambil oleh individu", example="Business Administration")
    university: str = Field(description="Nama universitas tempat individu belajar", example="Harvard University")
    country: str = Field(description="Negara tempat individu menyelesaikan pendidikannya", example="United States")

# Directory output untuk hasil JSON
output_dir = "results_2nd_turn/"
os.makedirs(output_dir, exist_ok=True)

# Loop utama pada setiap perusahaan
for index, row in tqdm(df_phase_1.iterrows(), total=df_phase_1.shape[0], desc="Processing Companies"):
    company = row["company"]
    members = []  # List untuk menyimpan semua hasil dari BOD & BOC perusahaan ini

    # Proses BOD (Dewan Direksi)
    try:
        bod_list = eval(row["bod"])  # Convert string representation of list to actual list
    except:
        bod_list = []  # Jika error parsing, buat list kosong

    for bod_member in bod_list:
        person_name = bod_member["name"]
        position = bod_member["position"]
        
        if person_name.strip():  # Pastikan nama valid sebelum query
            response = process_turn(
                query_prompt=f"Retrieve the bachelor education degree of {person_name}, who holds the position of {position} at {company}, from the company's report.",
                schema_class=EducationEntry,
                input_dir="data/JSON",
                output_dir="results_2nd_turn/",
                glob_path="*/*.json"
            )

            # Jika ada hasil yang valid, tambahkan ke daftar members
            if response and "data" in response:
                members.append(response["data"])

    # Proses BOC (Dewan Komisaris)
    try:
        boc_list = eval(row["boc"])  
    except:
        boc_list = []

    for boc_member in boc_list:
        person_name = boc_member["name"]
        position = boc_member["position"]
        
        if person_name.strip():  # Pastikan nama valid sebelum query
            response = process_turn(
                query_prompt=f"Retrieve the bachelor education degree of {person_name}, who holds the position of {position} at {company}, from the company's report.",
                schema_class=EducationEntry,
                input_dir="data/JSON",
                output_dir="results_2nd_turn/",
                glob_path="*/*.json"
            )

            # Jika ada hasil yang valid, tambahkan ke daftar members
            if response and "data" in response:
                members.append(response["data"])

    # Simpan hasil dalam file JSON per perusahaan **setelah semua BOD & BOC diproses**
    company_data = {
        "company": company,
        "members": members  # Semua hasil dari BOD & BOC sudah dikumpulkan di sini
    }
    
    company_filename = os.path.join(output_dir, f"{company}.json")
    with open(company_filename, "w", encoding="utf-8") as f:
        json.dump(company_data, f, indent=4, ensure_ascii=False)

    print(f"✅ Saved: {company_filename}")


Processing Companies:   0%|          | 0/8 [00:00<?, ?it/s]



> Entering new Invoke Agent chain...

Invoking: `retriever_tool` with `{'query': 'Tubagus Farash Akbar F education bachelor degree'}`


[]
Invoking: `retriever_tool` with `{'query': 'Tubagus Farash Akbar F education'}`


[]
Invoking: `retriever_tool` with `{'query': 'Tubagus Farash Akbar F'}`


[]I was unable to find any information regarding Tubagus Farash Akbar F's bachelor's degree or educational background in the company's reports. It's possible that this information may not be included in the available documents. If you have any other queries or need information on a different subject, please let me know!

> Finished chain.




> Entering new Invoke Agent chain...

Invoking: `retriever_tool` with `{'query': 'bachelor education degree Tubagus Farash Akbar F President Director'}`


[Document(id='c34e964e-c185-40f0-86d1-72bd94081525', metadata={'company': 'ID_ABBA_IR_2023', 'page': 51.0}, page_content='## Tubagus Farash Akbar F\n\n## Direktur Utama\n\nPresident Director\n\n<!-- image -->\n\nDasar Hukum Pengangkatan/ Legal Basis of Appointment Berdasarkan Akta/ Based on Deed\n\nNo. 16 Tanggal 27 Juni 2022\n\nNo. 16 Dated June 27, 2022\n\nKewarganegaraan/Nationality Indonesia\n\nTanggal Lahir/Date of Birth 10 Februari 1981/February 10, 1981\n\nUsia/Age 43 Tahun/43 Years Old\n\nDomisili/Domicile Jakarta\n\nBab 2\n\nLaporan Management\n\n## Direksi Board of Directors\n\n## Hubungan Afiliasi Affiliated Relations\n\n<!-- image -->\n\nTidak memiliki hubungan afiliasi dengan dengan anggota  Direksi  lainnya,  anggota  Dewan  Komisaris, pemegang  saham  utama  dan  pengendali,  baik langsung maupun tidak langsung sampa



> Entering new Invoke Agent chain...

Invoking: `retriever_tool` with `{'query': 'Tubagus Farash Akbar F education degree'}`


[Document(id='4020b24d-2c9f-4e15-8519-da2c40218a3c', metadata={'company': 'ID_ABDA_IR_2023', 'page': 179.0}, page_content='<!-- image -->\n\n<!-- image -->\n\n<!-- image -->\n\n<!-- image -->\n\n<!-- image -->\n\n## Riwayat Jabatan\n\nPosition History\n\n## Riwayat Pendidikan\n\nEducational Background\n\n## Riwayat Jabatan\n\nPosition History\n\n## Riwayat Pendidikan\n\nEducational Background\n\nSekilas Perusahaan Company Overview\n\nLaporan Manajemen Management Report\n\nAnalisis dan Pembahasan Manajemen Management and Discussion Analysis\n\nTata Kelola Perusahaan\n\nGood Corporate Governance\n\n## Tulus Pardamean\n\nAnggota / Member\n\nUsia\n\nAge\n\n38 Tahun / Years old\n\nKewarganegaraan Citizenship\n\nIndonesia\n\n## Cynthia I. Harninta\n\nAnggota / Member\n\nUsia\n\nAge\n\n32 Tahun / Years old\n\nKewarganegaraan Citizenship\n\nTanggung Jawab Sosial Peru



> Entering new Invoke Agent chain...

Invoking: `retriever_tool` with `{'query': 'Tubagus Farash Akbar F education bachelor degree'}`


[Document(id='8a2ececa-649a-460c-b209-9831b2e974ef', metadata={'company': 'ID_ACRO_IR_2023', 'page': 54.0}, page_content='<!-- image -->\n\nLaporan Manajemen\n\nManagement Report\n\n<!-- image -->\n\n## Profil Unit Audit Internal\n\n## PROFILE OF INTERNAL AUDIT UNIT\n\n<!-- image -->\n\n## Kewarganegaraan\n\n## Citizenship\n\nWarga Negara Indonesia berusia 33 tahun.\n\nIndonesian Citizen, 33 years old.\n\n## Riwayat Pendidikan\n\n## Educational background\n\npendidikan di Universitas Islam\n\nMenyelesaikan Indonesia, Yogyakarta.\n\n## Pengalaman Bekerja\n\n## Work experience\n\n- · Accounting &amp; Tax Manager di PT Samcro Hyosung Adilestari\n- · Chief Accounting &amp; Tax di PT Keramindo Kimia\n- · Finance Manager di PT Jin Woo Engineering\n- · Chief Accounting &amp; Tax di PT Ikatama Citra Mandiri\n- · Finance Manager di PT Duta Korina\n- · Chief A

...

Processing Companies: 100%|██████████| 11/11 [07:37<00:00, 41.56s/it]




> Entering new Invoke Agent chain...

Invoking: `retriever_tool` with `{'query': 'Bachelor education degree of Ishak Reza, Director'}`


[]It seems that I couldn't find any information regarding the bachelor's education degree of Ishak Reza in the company's report. If there are specific details or other related queries you'd like to explore, please let me know!

> Finished chain.




> Entering new Invoke Agent chain...

Invoking: `retriever_tool` with `{'query': 'Ishak Reza, Director, bachelor education degree'}`


[Document(id='c048d947-21d6-4569-bc2c-0ab36a00238e', metadata={'company': 'ID_ABBA_IR_2023', 'page': 53.0}, page_content='## Ishak Reza Direktur Director\n\nDasar Hukum Pengangkatan/ Legal Basis of Appointment Berdasarkan Akta/ Based on Deed\n\n<!-- image -->\n\nNo. 11 Tanggal 16 Desember 2022 No. 11 Dated Desember 16, 2022\n\nKewarganegaraan/Nationality Indonesia\n\nTanggal Lahir/Date of Birth 30 Oktober 1986/October 30, 1986\n\nUsia/Age 37 Tahun/37 Years Old\n\nDomisili/Domicile Jakarta\n\nBab 2\n\nLaporan Management\n\n## Hubungan Afiliasi\n\n## Affiliated Relations\n\nTidak memiliki hubungan afiliasi dengan dengan anggota  Direksi  lainnya,  anggota  Dewan  Komisaris, pemegang  saham  utama  dan  pengendali,  baik langsung maupun tidak langsung sampai kepada pemilik individu.\n\nHas  no  affiliation  with  other  members  of  the Board of Director

Processing Companies:   0%|          | 0/8 [16:52<?, ?it/s]


KeyboardInterrupt: 